In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/Installment/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(200, 2)


,label,split_text
0,0,那 时间 肯定 够 了
1,1,估计 不行
2,2,诶 你 再 说 一次
3,0,行
4,0,好 的
5,0,这样 应该 没 问题 了
6,0,那 我 会 按照 分期 按时 还款 的
7,0,我 会 按时 还款 的
8,0,我 会 按照 这个 准时 还款 的
9,0,没 问题


In [3]:
data.split_text.iloc[42]

'你 再说 一遍'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<200x856 sparse matrix of type '<class 'numpy.float64'>'
	with 1810 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

1.0


In [6]:
print(clf.predict(phrase))

[0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2]


# logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

1.0


In [9]:
print(log_r.predict(phrase))

[0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2
 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 2 0 1 2 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2]


# LightGBM

In [10]:
import lightgbm as lgb

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [12]:
onelabels

array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [14]:
lables

array([[1, 1, 1]])

In [15]:
data.label.values

array([0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [16]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.355
[10]	valid_0's multi_error: 0.34
[15]	valid_0's multi_error: 0.345
[20]	valid_0's multi_error: 0.345
[25]	valid_0's multi_error: 0.345
[30]	valid_0's multi_error: 0.345
[35]	valid_0's multi_error: 0.345
[40]	valid_0's multi_error: 0.345
[45]	valid_0's multi_error: 0.345
[50]	valid_0's multi_error: 0.335
[55]	valid_0's multi_error: 0.335
[60]	valid_0's multi_error: 0.335
[65]	valid_0's multi_error: 0.335
[70]	valid_0's multi_error: 0.335
[75]	valid_0's multi_error: 0.33
[80]	valid_0's multi_error: 0.33
[85]	valid_0's multi_error: 0.33
[90]	valid_0's multi_error: 0.33
[95]	valid_0's multi_error: 0.33
[100]	valid_0's multi_error: 0.33
[105]	valid_0's multi_error: 0.33
[110]	valid_0's multi_error: 0.33
[115]	valid_0's multi_error: 0.33
[120]	valid_0's multi_error: 0.33
[125]	valid_0's multi_error: 0.33
[130]	valid_0's multi_error: 0.33
[135]	valid_0's multi_error: 0.33
[140]	valid_0's multi_erro

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [17]:
print(lgbm_model.predict(phrase))

[[ 0.70930292  0.17125217  0.19952586]
 [ 0.54299947  0.25342914  0.20836501]
 [ 0.03353246  0.31266149  0.3355672 ]
 [ 0.54299947  0.25342914  0.20836501]
 [ 0.54633707  0.43578085  0.04452865]
 [ 0.70930292  0.17125217  0.19952586]
 [ 0.2518907   0.63614741  0.04507387]
 [ 0.13999585  0.44274147  0.09841601]
 [ 0.2518907   0.68751803  0.04310711]
 [ 0.54299947  0.25342914  0.20836501]
 [ 0.70930292  0.22155302  0.04226858]
 [ 0.54633707  0.43578085  0.04452865]
 [ 0.54299947  0.25342914  0.20836501]
 [ 0.11891667  0.94489459  0.00984539]
 [ 0.0197749   0.39627999  0.49937404]
 [ 0.54299947  0.25342914  0.20836501]
 [ 0.54299947  0.25342914  0.20836501]
 [ 0.15446479  0.50306913  0.28219884]
 [ 0.11750092  0.94489459  0.00984539]
 [ 0.53962708  0.25342914  0.20836501]
 [ 0.70930292  0.17125217  0.19952586]
 [ 0.11750092  0.94489459  0.00984539]
 [ 0.0197749   0.36189656  0.54550673]
 [ 0.11782431  0.4995965   0.39216103]
 [ 0.01826332  0.58134092  0.39639267]
 [ 0.00999198  0.90983783

In [18]:
phrase

<200x856 sparse matrix of type '<class 'numpy.float64'>'
	with 1810 stored elements in Compressed Sparse Row format>

# Save

In [20]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/Installment/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/Installment/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/Installment/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/Installment/Lgbm.pickle", "wb"))

# Test

In [22]:
test = phrase_vectorizer.transform(['我 真的 没有 钱 呀'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])


In [23]:
clf.predict_proba(test) # linear svc

array([[ 0.0133909 ,  0.97495891,  0.01165019]])

In [24]:
log_r.predict_proba(test) # logistic

array([[ 0.12950896,  0.7129067 ,  0.15758434]])

In [25]:
lgbm_model.predict(test) # light gbm

array([[ 0.00999198,  0.90983783,  0.09331524]])

# Labeling

In [26]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.97495890951
